# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraB_modes.ipynb
- template script for analyzing modes spectra for sumatra

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from lib_seis import locations_and_tags
from lib_seis import get_dist_az
from lib_seis import response
from lib_seis import station_map_and_table
from lib_seis import sumatra_event
from lib_seis import wf_fft
from lib_seis import w2fstack
from obspy import read
from obspy import Stream

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

class SEED_ID_Error(Exception):
    pass

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for i, item in enumerate(event):
    print(f'{item} = {event[item]}')
print()    

In [ ]:
# setting other data parameters

resample_freq = 1
data_dir      = 'datawf/sumatra_LHZ_raw'

In [ ]:
# read downloaded LHZ acceleration data for selected station locations

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# plotting and listing station information corresponding to selected waveforms

# paste your list of selected waveforms from lab_sumatraB here -
picked_waveforms = ['G.CAN..LHZ'    ,
                    'II.NNA.00.LHZ' ,
                    'II.SUR.00.LHZ' ,
                    'IU.BBSR.00.LHZ']

st_select = Stream()

for seed_id in picked_waveforms:
    st_pick = st.select(id=seed_id)
    if not st_pick:
        raise SEED_ID_Error(f'{seed_id} not in inventory')
    st_select += st_pick

event_path = f'{data_dir}/event.xml'
station_map_and_table(st_select, event_path)

In [ ]:
# computing and creating a list of the amplitude spectra of the seismograms for the selected station locations

amps  = []
freqs = []

Nyquist_frequency = resample_freq/2

c = 0
for tr_select in st_select:

    tr = tr_select.copy()

    c = c + 1
    print(f'{c}. {tr.id}')

    tr.detrend('constant')
    tr.detrend('linear')
    tr.taper(max_percentage=0.2, type="cosine")

    fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)

    # function response() is in lib_seis.py and uses the ObsPy function
    # get_evalresp_response_for_frequencies which uses the full response file

    Ia = response(tr, fft_freq, output='ACC', start_stage=None, end_stage=None)
    C   = fft_amp * np.exp( 1j * fft_phase )
    Xa  = np.divide(C,Ia)
    [fft_amp , fft_phase , fft_freq ] = [abs(Xa), np.angle(Xa), fft_freq]

    freqs.append(fft_freq)
    amps.append(fft_amp)

In [ ]:
# example of stacked spectra

f1 = 0.2 * 1E-3
f2 = 10  * 1E-3
n  = 100000

amp_stack, freq, amplitudes = w2fstack(freqs,amps,f1,f2,n,stack='median')                                                     

plt.figure()
plt.plot(freq*1E3, amp_stack, 'b') 
plt.title('Amplitude spectra stack ({f1*1E3} - {f2*1E3} mHz)')
plt.xlabel('Frequency [mHz]')
plt.ylabel('Amplitude (ms$^-$$^2$Hz$^-$$^1$)')
plt.xlim(f1*1E3, f2*1E3)
plt.show()